In [1]:
import pickle
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold

In [2]:
with open("test7", "rb") as f:   # Unpickling
    list_dfs_signif = pickle.load(f)

In [3]:
list_dfs_signif[665][1:]

(1, 0, 0)

En esta lista tenemos tuplas con las matrices que hay por cada preproceso, trozo y canal. Estas tuplas tienen la forma:
* DataFrame con la matriz correspondiente
* int que indica el tipo de preproceso
* int que indica el trozo de la grabación
* int que indica el canal

Deberíamos tener 4x35x19=2660 matrices, pero tenemos solo 2652. Esto se debe a que hay 8 casos en los que ninguna característica muestra diferencias estadísticamente significativas.

Veamos qué 8 combinaciones faltan:

1. p:1, t:7, c:1
2. p:1, t:11, c:0
3. p:1, t:19, c:15
4. p:1, t:23, c:1
5. p:1, t:24, c:1
6. p:1, t:27, c:1
7. p:1, t:27, c:7
8. p:3, t:34, c:18

In [11]:
#tuplas = [x[1:] for x in list_dfs_signif]

#df_tuplas = pd.DataFrame(tuplas, columns =['preproceso', 'trozo', 'canal'])
#df_tuplas[(df_tuplas['preproceso'] == 3) & (df_tuplas['trozo'] == 34)]


In [30]:
#item = list_dfs_signif[0]
#new_df = item[0].iloc[:,:-1]
#df = new_df.add_prefix(str(0)+'_'+str(0)+'_'+str(0)+'_')
#df


,0_0_0_8,0_0_0_11,0_0_0_12,0_0_0_14,0_0_0_18,0_0_0_19,0_0_0_25,0_0_0_26,0_0_0_27,0_0_0_40
0,1.240511,0.766067,-0.815781,-0.772619,-0.143892,0.983624,0.039024,0.405333,-0.671016,1.983540
1,0.048250,0.879066,-0.452122,-0.125923,0.749749,-1.558572,-1.628951,1.526797,-0.395786,0.066260
2,-0.780672,0.595220,0.635574,-0.097670,0.795935,-0.892255,-0.421196,0.691743,0.265807,-0.325473
3,-0.408717,0.700499,0.630771,-0.595176,1.115828,-0.457313,-0.350016,1.008267,-0.036952,-0.462469
4,0.810423,0.950024,-1.155103,-0.881343,-0.204638,0.387990,-0.959636,0.958041,-1.654557,1.803558
...,...,...,...,...,...,...,...,...,...,...
116,1.687058,-0.242105,-0.755685,-0.814659,-0.863397,1.226115,1.098856,-0.289425,-0.394972,-0.054449
117,-0.129228,-1.462917,-0.639618,0.182474,-1.330027,1.150737,0.350494,-1.697230,0.095881,-0.781108
118,1.787035,-0.966912,-0.111339,-0.265635,-1.016808,1.156707,2.176186,-0.746613,0.119894,0.364161
119,2.473547,0.659886,-1.340117,-0.933288,0.060197,1.187555,0.725741,0.067712,-1.734177,2.553461


In [4]:
#Listas para guardar listas
lista_prep = []
lista_troz = []
lista_chan = []
# inicializamos los parámetros
prep = 0
troz = 0
chan = 0
# iteramos sobre todas las tuplas
for item in list_dfs_signif:
    # cogemos las características de la nueva tupla
    new_df = item[0].iloc[:,:-1]
    new_prep = item[1]
    new_troz = item[2]
    new_chan = item[3]
    # Comprobamos que el preproceso nuevo y el anterior sean iguales
    if (new_prep == prep):
        # comprobamos que el trozo nuevo y el anterior sean iguales
        if (new_troz == troz):
            # Añadimos el prep, el trozo y el canal el trozo a las columnas:
            df = new_df.add_prefix(str(new_prep)+'_'+str(new_troz)+'_'+str(new_chan)+'_')
            # Guardamos la matriz en la lista correspondiente
            lista_chan.append(df)
            prep = new_prep
            troz = new_troz
            chan = new_chan
        # Si el trozo cambia:
        else:
            # añadimos la lista de canales a la lista de trozos
            lista_troz.append(lista_chan)
            # inicializamos la lista de los canales
            lista_chan = []
            # Añadimos el prep, el trozo y el canal a las columnas:
            df = new_df.add_prefix(str(new_prep)+'_'+str(new_troz)+'_'+str(new_chan)+'_')
            lista_chan.append(df)
            prep = new_prep
            troz = new_troz
            chan = new_chan
    # Si el preproceso cambia:
    else:
        # Añadimos la lista de canales a la lista de trozos
        lista_troz.append(lista_chan)
        # Añadimos la lista de trozos a la lista de preprocesos
        lista_prep.append(lista_troz)
        # Inicializamos la lista de los canales
        lista_chan = []
        # Inicializamos la lista de los trozos
        lista_troz = []
        # Añadimos el prep, el trozo y el canal a las columnas:
        df = new_df.add_prefix(str(new_prep)+'_'+str(new_troz)+'_'+str(new_chan)+'_')
        lista_chan.append(df)
        prep = new_prep
        troz = new_troz
        chan = new_chan
lista_troz.append(lista_chan)
lista_prep.append(lista_troz)

In [5]:
lista_prep[3][0][0]

,3_0_0_8,3_0_0_9,3_0_0_11,3_0_0_12,3_0_0_18,3_0_0_20,3_0_0_21,3_0_0_32,3_0_0_33
0,1.704916,-1.686117,2.186936,-1.496778,2.809643,-0.246423,-0.412794,-2.115272,-0.346690
1,0.605762,-0.908847,1.467244,-1.052218,0.870472,-0.714113,2.282763,-1.244701,1.110818
2,-1.112867,-0.510552,-0.797458,1.308273,-0.432740,1.292090,-0.491504,0.496441,-0.612297
3,-0.470675,-1.163185,-1.358602,0.880269,-0.910214,-0.712566,-0.949452,1.367013,-0.300361
4,0.364063,0.205878,-0.070887,-0.780964,-0.626521,0.005577,-0.076488,-0.374130,-0.548615
...,...,...,...,...,...,...,...,...,...
116,-0.342852,0.559681,-0.375502,0.293202,-0.426680,1.303748,0.139197,0.061156,-0.387689
117,0.637023,-0.436398,1.020901,-0.025928,1.447409,-0.541164,-1.350157,-0.809415,-0.885604
118,0.575906,-0.685806,0.512701,-0.410184,-0.234659,-0.808319,-1.239758,-0.809415,-0.663593
119,3.883303,1.961690,0.034831,-2.202284,-0.760739,1.549429,-0.585547,-0.809415,-0.264197


Ahora tengo una lista con 4 listas (cada uno de los preprocesos). Cada una de estas listas tiene a su vez tantas listas como trozos (35) y, cada una de las listas de trozos contiene una lista por cada canal con las matrices DES.
Hay que tener en cuenta, que al haber combinaciones que no se dan (escritas arriba) no todas las posiciones de las matrices coinciden con las combinaciones reales, por lo que debemos extraer las combinaciones del nombre de las columnas y no de las posiciones.

El siguiente paso es generar nuevas matrices en las listas de trozos, juntando las matrices de canales de 3 en 3, como hicimos con el total de matrices.

In [5]:
df_1 = lista_prep[1][7][0]
df_2 = lista_prep[0][0][1] 
df_3 = lista_prep[0][0][2]
lab_chan_1 = int(df_1.columns[0].split('_')[2])
lab_chan_2 = int(df_2.columns[0].split('_')[2])
lab_chan_3 = int(df_3.columns[0].split('_')[2])
df4 = pd.concat([df_1,df_2,df_3], axis=1)
list_lab_chan =  [(lab_chan_1,lab_chan_2,lab_chan_3)]*121
df4['channels'] = list_lab_chan
df4.shape[0]

121

In [6]:
def conversor_num_to_chan(num):
    if num == 0:
        a = 'Fp1'
    elif num == 1:
        a = 'Fp2'
    elif num == 2:
        a = 'F3'
    elif num == 3:
        a = 'F4'
    elif num == 4:
        a = 'C3'
    elif num == 5:
        a = 'C4'
    elif num == 6:
        a = 'P3'
    elif num == 7:
        a = 'P4'
    elif num == 8:
        a = 'O1'
    elif num == 9:
        a = 'O2'
    elif num == 10:
        a = 'F7'
    elif num == 11:
        a = 'F8'
    elif num == 12:
        a = 'T7'
    elif num == 13:
        a = 'T8'
    elif num == 14:
        a = 'P7'
    elif num == 15:
        a = 'P8'
    elif num == 16:
        a = 'Fz'
    elif num == 17:
        a = 'Cz'
    elif num == 18:
        a = 'Pz'
    else:
        print('error de canales')
    return a

In [7]:
# Añadida la etiqueta de cada
lista_new_preproceso = []
lista_total = []
for list_trozos in lista_prep:
    lista_new_trozos = []
    for list_channels in list_trozos:
        lista_new_matrices = []
        for i in range(len(list_channels)):
            for j in range(len(list_channels)):
                for k in range(len(list_channels)):
                    if (j>i) and (k>j):
                        label = list_dfs_signif[0][0].iloc[:,-1]
                        df1 = list_channels[i]
                        df2 = list_channels[j]
                        df3 = list_channels[k]
                        # cogemos los números reales de los canales
                        lab_chan_1 = int(df1.columns[0].split('_')[2])
                        lab_chan_2 = int(df2.columns[0].split('_')[2])
                        lab_chan_3 = int(df3.columns[0].split('_')[2])
                        # los convertimos al nombre del canal
                        lab_chan_1 = conversor_num_to_chan(lab_chan_1)
                        lab_chan_2 = conversor_num_to_chan(lab_chan_2)
                        lab_chan_3 = conversor_num_to_chan(lab_chan_3)
                        # Unimos los 3 dfs
                        df4 = pd.concat([df1,df2,df3], axis=1)
                        # Creamos las columnas con los canales y las etiquetas
                        list_lab_chan =  [(lab_chan_1,lab_chan_2,lab_chan_3)]*df4.shape[0]
                        df4['Label'] = label
                        df4['Channels'] = list_lab_chan
                        df4['chunk'] = [int(df2.columns[0].split('_')[1])]*df4.shape[0]
                        df4['prep'] = [int(df2.columns[0].split('_')[0])]*df4.shape[0]
                        lista_new_matrices.append(df4)
                        lista_total.append(df4)
        lista_new_trozos.append(lista_new_matrices)
    lista_new_preproceso.append(lista_new_trozos)

In [26]:
with open("test10", "wb") as fp:   #Pickling
    pickle.dump(lista_new_preproceso, fp)

In [8]:
with open("3canales", "wb") as fp:   #Pickling
    pickle.dump(lista_total, fp)

In [25]:
lista_new_preproceso[0][0][0]

,0_0_0_8,0_0_0_11,0_0_0_12,0_0_0_14,0_0_0_18,0_0_0_19,0_0_0_25,0_0_0_26,0_0_0_27,0_0_0_40,...,0_0_2_14,0_0_2_19,0_0_2_21,0_0_2_26,0_0_2_27,0_0_2_33,Label,Channels,chunk,prep
0,1.240511,0.766067,-0.815781,-0.772619,-0.143892,0.983624,0.039024,0.405333,-0.671016,1.983540,...,-0.955431,0.838717,-0.015190,0.491016,-1.036441,-0.032399,1,"(Fp1, Fp2, F3)",0,0
1,0.048250,0.879066,-0.452122,-0.125923,0.749749,-1.558572,-1.628951,1.526797,-0.395786,0.066260,...,-0.843946,-1.298227,-0.749964,1.857206,-0.940285,-0.584836,1,"(Fp1, Fp2, F3)",0,0
2,-0.780672,0.595220,0.635574,-0.097670,0.795935,-0.892255,-0.421196,0.691743,0.265807,-0.325473,...,-0.136965,-1.381840,-0.457282,1.239203,-0.134332,-0.557357,1,"(Fp1, Fp2, F3)",0,0
3,-0.408717,0.700499,0.630771,-0.595176,1.115828,-0.457313,-0.350016,1.008267,-0.036952,-0.462469,...,0.225994,-1.170758,-0.803588,0.456744,0.988484,-0.581247,1,"(Fp1, Fp2, F3)",0,0
4,0.810423,0.950024,-1.155103,-0.881343,-0.204638,0.387990,-0.959636,0.958041,-1.654557,1.803558,...,-0.071052,0.614745,-0.256294,-0.636911,0.959070,-0.217839,1,"(Fp1, Fp2, F3)",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,1.687058,-0.242105,-0.755685,-0.814659,-0.863397,1.226115,1.098856,-0.289425,-0.394972,-0.054449,...,-0.899792,1.210511,0.277901,-0.547167,-0.613488,0.358885,0,"(Fp1, Fp2, F3)",0,0
117,-0.129228,-1.462917,-0.639618,0.182474,-1.330027,1.150737,0.350494,-1.697230,0.095881,-0.781108,...,-0.456234,1.220189,1.177642,-1.334235,-0.718751,0.390078,0,"(Fp1, Fp2, F3)",0,0
118,1.787035,-0.966912,-0.111339,-0.265635,-1.016808,1.156707,2.176186,-0.746613,0.119894,0.364161,...,0.331886,1.061303,0.250066,-1.161357,-0.030521,0.300965,0,"(Fp1, Fp2, F3)",0,0
119,2.473547,0.659886,-1.340117,-0.933288,0.060197,1.187555,0.725741,0.067712,-1.734177,2.553461,...,-1.053487,1.240352,0.401933,-0.628896,-0.817467,0.608387,0,"(Fp1, Fp2, F3)",0,0


In [19]:
lista_total[0]

,0_0_0_8,0_0_0_11,0_0_0_12,0_0_0_14,0_0_0_18,0_0_0_19,0_0_0_25,0_0_0_26,0_0_0_27,0_0_0_40,...,0_0_2_14,0_0_2_19,0_0_2_21,0_0_2_26,0_0_2_27,0_0_2_33,Label,Channels,chunk,prep
0,1.240511,0.766067,-0.815781,-0.772619,-0.143892,0.983624,0.039024,0.405333,-0.671016,1.983540,...,-0.955431,0.838717,-0.015190,0.491016,-1.036441,-0.032399,1,"(Fp1, Fp2, F3)",0,0
1,0.048250,0.879066,-0.452122,-0.125923,0.749749,-1.558572,-1.628951,1.526797,-0.395786,0.066260,...,-0.843946,-1.298227,-0.749964,1.857206,-0.940285,-0.584836,1,"(Fp1, Fp2, F3)",0,0
2,-0.780672,0.595220,0.635574,-0.097670,0.795935,-0.892255,-0.421196,0.691743,0.265807,-0.325473,...,-0.136965,-1.381840,-0.457282,1.239203,-0.134332,-0.557357,1,"(Fp1, Fp2, F3)",0,0
3,-0.408717,0.700499,0.630771,-0.595176,1.115828,-0.457313,-0.350016,1.008267,-0.036952,-0.462469,...,0.225994,-1.170758,-0.803588,0.456744,0.988484,-0.581247,1,"(Fp1, Fp2, F3)",0,0
4,0.810423,0.950024,-1.155103,-0.881343,-0.204638,0.387990,-0.959636,0.958041,-1.654557,1.803558,...,-0.071052,0.614745,-0.256294,-0.636911,0.959070,-0.217839,1,"(Fp1, Fp2, F3)",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,1.687058,-0.242105,-0.755685,-0.814659,-0.863397,1.226115,1.098856,-0.289425,-0.394972,-0.054449,...,-0.899792,1.210511,0.277901,-0.547167,-0.613488,0.358885,0,"(Fp1, Fp2, F3)",0,0
117,-0.129228,-1.462917,-0.639618,0.182474,-1.330027,1.150737,0.350494,-1.697230,0.095881,-0.781108,...,-0.456234,1.220189,1.177642,-1.334235,-0.718751,0.390078,0,"(Fp1, Fp2, F3)",0,0
118,1.787035,-0.966912,-0.111339,-0.265635,-1.016808,1.156707,2.176186,-0.746613,0.119894,0.364161,...,0.331886,1.061303,0.250066,-1.161357,-0.030521,0.300965,0,"(Fp1, Fp2, F3)",0,0
119,2.473547,0.659886,-1.340117,-0.933288,0.060197,1.187555,0.725741,0.067712,-1.734177,2.553461,...,-1.053487,1.240352,0.401933,-0.628896,-0.817467,0.608387,0,"(Fp1, Fp2, F3)",0,0


In [8]:
lista_total[17]

,0_0_0_8,0_0_0_11,0_0_0_12,0_0_0_14,0_0_0_18,0_0_0_19,0_0_0_25,0_0_0_26,0_0_0_27,0_0_0_40,...,0_0_3_13,0_0_3_14,0_0_3_19,0_0_3_22,0_0_3_25,0_0_3_27,Label,Channels,chunk,prep
0,1.240511,0.766067,-0.815781,-0.772619,-0.143892,0.983624,0.039024,0.405333,-0.671016,1.983540,...,0.607957,1.243875,-1.080865,-0.501746,-0.648352,0.965141,1,"(Fp1, F3, F4)",0,0
1,0.048250,0.879066,-0.452122,-0.125923,0.749749,-1.558572,-1.628951,1.526797,-0.395786,0.066260,...,-0.684684,-0.927169,-0.580802,-0.662303,-0.828024,-1.119804,1,"(Fp1, F3, F4)",0,0
2,-0.780672,0.595220,0.635574,-0.097670,0.795935,-0.892255,-0.421196,0.691743,0.265807,-0.325473,...,0.274507,0.281075,-1.090121,-0.674115,0.138192,0.457628,1,"(Fp1, F3, F4)",0,0
3,-0.408717,0.700499,0.630771,-0.595176,1.115828,-0.457313,-0.350016,1.008267,-0.036952,-0.462469,...,1.260004,-0.129975,0.130002,-0.252097,0.549037,0.940070,1,"(Fp1, F3, F4)",0,0
4,0.810423,0.950024,-1.155103,-0.881343,-0.204638,0.387990,-0.959636,0.958041,-1.654557,1.803558,...,1.581665,-0.219172,0.717799,0.535984,-0.137162,0.992356,1,"(Fp1, F3, F4)",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,1.687058,-0.242105,-0.755685,-0.814659,-0.863397,1.226115,1.098856,-0.289425,-0.394972,-0.054449,...,-0.271113,-0.494236,1.368248,1.164193,0.957628,0.086420,0,"(Fp1, F3, F4)",0,0
117,-0.129228,-1.462917,-0.639618,0.182474,-1.330027,1.150737,0.350494,-1.697230,0.095881,-0.781108,...,-0.800246,-0.560717,1.555576,1.127679,0.863018,-0.913933,0,"(Fp1, F3, F4)",0,0
118,1.787035,-0.966912,-0.111339,-0.265635,-1.016808,1.156707,2.176186,-0.746613,0.119894,0.364161,...,-0.567946,-0.142763,0.929768,0.896926,1.697182,-0.316794,0,"(Fp1, F3, F4)",0,0
119,2.473547,0.659886,-1.340117,-0.933288,0.060197,1.187555,0.725741,0.067712,-1.734177,2.553461,...,-0.787951,-0.807115,1.483388,1.372443,1.233570,-0.527079,0,"(Fp1, F3, F4)",0,0


1. p:1, t:7, c:1
2. p:1, t:11, c:0
3. p:1, t:19, c:15
4. p:1, t:23, c:1
5. p:1, t:24, c:1
6. p:1, t:27, c:1
7. p:1, t:27, c:7
8. p:3, t:34, c:18

In [9]:
from math import factorial

In [10]:
def combinaciones(n,r):
    num = factorial(n)
    den = factorial(r) * factorial(n-r)
    res = num/den
    return int(res)
print(35*combinaciones(19,3)*4)
print(len(lista_total))

print(35*combinaciones(19,3) + \
      29*combinaciones(19,3) + \
       5*combinaciones(18,3) + \
       1*combinaciones(17,3) + \
      35*combinaciones(19,3) + \
      34*combinaciones(19,3) + \
       1*combinaciones(18,3))

135660
134453
134453


In [8]:
# código anterior
# lista_new_preproceso = []
# for list_trozos in lista_prep:
#     lista_new_trozos = []
#     for list_channels in list_trozos:
#         lista_new_matrices = []
#         for i in range(len(list_channels)):
#             for j in range(len(list_channels)):
#                 for k in range(len(list_channels)):
#                     if (j>i) and (k>j):
#                         label = list_dfs_signif[0][0].iloc[:,-1]
#                         df1 = list_channels[i]
#                         df2 = list_channels[j]
#                         df3 = list_channels[k]
#                         df4 = pd.concat([df1,df2,df3], axis=1)
#                         df4['Label'] = label
#                         lista_new_matrices.append(df4)
#                 lista_new_trozos.append(lista_new_matrices)
#         lista_new_preproceso.append(lista_new_trozos)

In [26]:
lista_total[2923]#.iloc[:,:-4]

,0_3_0_4,0_3_0_5,0_3_0_8,0_3_0_11,0_3_0_12,0_3_0_13,0_3_0_17,0_3_0_18,0_3_0_19,0_3_0_25,...,0_3_18_25,0_3_18_26,0_3_18_27,0_3_18_31,0_3_18_34,0_3_18_39,Label,Channels,chunk,prep
0,0.519570,0.077723,1.371327,1.049652,-0.870022,-0.574101,-0.470090,0.536536,0.920052,0.258113,...,1.329624,-1.438383,0.376663,-0.526662,-0.606029,-0.468123,1,"(Fp1, Fp2, Pz)",3,0
1,0.061046,-0.437965,0.191179,0.613526,-0.008014,0.098525,-0.936895,0.583652,-1.513165,-1.617285,...,-0.937856,1.143989,0.243485,-0.466848,-0.556975,-0.349176,1,"(Fp1, Fp2, Pz)",3,0
2,0.736798,0.182651,-0.875751,0.572570,0.524775,-0.128938,-0.796422,0.680533,-0.753106,-0.131111,...,0.311614,-0.478474,0.884428,1.501949,1.407291,-0.450949,1,"(Fp1, Fp2, Pz)",3,0
3,-0.390179,-0.032495,-0.578167,1.020264,0.241415,-0.707713,-0.940686,1.282480,-0.501501,-0.268046,...,-0.896487,1.214794,0.629082,-0.485686,-0.530214,-0.393427,1,"(Fp1, Fp2, Pz)",3,0
4,5.214783,8.329307,1.330019,1.228313,-1.707008,-0.953522,0.198331,-0.471319,0.386448,-1.321234,...,1.230800,-0.926384,0.712900,0.810051,0.883715,-0.383396,1,"(Fp1, Fp2, Pz)",3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,-0.534306,-0.279317,2.194526,-1.034856,-0.834632,-0.421890,1.343905,-1.095124,1.280286,0.892717,...,0.265142,-0.528479,-0.058604,-0.762929,-0.816686,-0.365858,0,"(Fp1, Fp2, Pz)",3,0
117,-0.046567,-0.477235,-0.006940,-1.788462,-0.470410,-0.458304,1.984568,-1.336459,1.131212,0.025688,...,0.259680,-1.116390,-0.658321,-0.513826,-0.598819,-0.450708,0,"(Fp1, Fp2, Pz)",3,0
118,1.402063,-0.032226,1.723613,-0.525266,-0.033915,-0.105364,0.720433,-0.786208,1.104477,2.142143,...,1.200563,-1.123164,-0.670997,-0.344545,-0.459427,1.322743,0,"(Fp1, Fp2, Pz)",3,0
119,-0.480300,-0.497838,2.988670,0.724997,-1.411617,-1.299051,-0.348466,0.419288,1.229655,0.669945,...,0.762706,-0.445807,-1.069747,-0.845666,-0.913886,1.561031,0,"(Fp1, Fp2, Pz)",3,0


En esta parte podríamos hacerlo mejor y poner directamente a qué sensores corresponde, por ejemplo 0-0-1, 23-24-20... y después convertirlo con la función de número-nombre.

In [33]:
# lista_total contiene todas las matrices necesarias, una tras otra 134453.
list_results = []
# for n_prep, prep in enumerate(lista_new_preproceso):
#     for n_troz, troz in enumerate(prep):
#         for n_chan, mat in enumerate(troz):
for mat in lista_total:
    for eta in [0.01, 0.1, 0.5, 1]:
        for gamma in [0, 1]:
            for max_depth in [2, 3, 6, 12, 24]:
                print('prep:', mat.iloc[0,-1],\
                      'troz:', mat.iloc[0,-2],\
                      'chan:', mat.iloc[0,-3],)
                      #'eta:', eta,'gamma:', gamma,'max_depth:', max_depth)
                # Cogemos los datos
                data_dm = mat.iloc[:,:-4].to_numpy()
                label_dm = mat['Label'].to_numpy()
                dtrain = xgb.DMatrix(data_dm, label=label_dm)
                param = {"max_depth": max_depth, "eta": eta , "gamma": gamma, "objective": "binary:logistic"}
                num_round = 30
                k_folds = StratifiedKFold(n_splits=5)
                res = xgb.cv(
                        param,
                        dtrain,
                        num_round,
                        stratified=True,
                        folds= k_folds,
                        metrics={"error"},
                        seed=0,
                        verbose_eval=0
                    )
                min_row = res[res['test-error-mean'] == res['test-error-mean'].min()].iloc[0,:]
                min_error = round(min_row.iloc[2],4)
                std_error = round(min_row.iloc[3], 4)
                list_results.append((mat.iloc[0,-1], mat.iloc[0,-2], mat.iloc[0,-3], eta, gamma, max_depth, 1-min_error, std_error))
                print('Accuracy máximo obtenido:', 1-min_error, '+-', std_error)


prep: 0 troz: 0 chan: ('Fp1', 'Fp2', 'F3')
Accuracy máximo obtenido: 0.6859999999999999 +- 0.0932
prep: 0 troz: 0 chan: ('Fp1', 'Fp2', 'F3')
Accuracy máximo obtenido: 0.661 +- 0.1278
prep: 0 troz: 0 chan: ('Fp1', 'Fp2', 'F3')
Accuracy máximo obtenido: 0.62 +- 0.124
prep: 0 troz: 0 chan: ('Fp1', 'Fp2', 'F3')
Accuracy máximo obtenido: 0.62 +- 0.124
prep: 0 troz: 0 chan: ('Fp1', 'Fp2', 'F3')
Accuracy máximo obtenido: 0.62 +- 0.124
prep: 0 troz: 0 chan: ('Fp1', 'Fp2', 'F3')
Accuracy máximo obtenido: 0.6859999999999999 +- 0.0932
prep: 0 troz: 0 chan: ('Fp1', 'Fp2', 'F3')
Accuracy máximo obtenido: 0.661 +- 0.1278
prep: 0 troz: 0 chan: ('Fp1', 'Fp2', 'F3')
Accuracy máximo obtenido: 0.62 +- 0.124
prep: 0 troz: 0 chan: ('Fp1', 'Fp2', 'F3')
Accuracy máximo obtenido: 0.62 +- 0.124
prep: 0 troz: 0 chan: ('Fp1', 'Fp2', 'F3')
Accuracy máximo obtenido: 0.62 +- 0.124
prep: 0 troz: 0 chan: ('Fp1', 'Fp2', 'F3')
Accuracy máximo obtenido: 0.7027 +- 0.0841
prep: 0 troz: 0 chan: ('Fp1', 'Fp2', 'F3')
Accurac

In [34]:
list_results.sort(key=lambda x: x[6], reverse=True)
df_results_xgb = pd.DataFrame(list_results, columns = ['preproceso', 'trozo', 'canal', 'eta', 'gamma',
                                                      'max_depth', 'accuracy', 'std_error'])
# Actual recortada
df_results_xgb.head(50)

,preproceso,trozo,canal,eta,gamma,max_depth,accuracy,std_error
0,0,0,"(F3, F4, T7)",0.10,0,3,0.7933,0.0590
1,0,0,"(F3, F4, T7)",0.10,1,3,0.7933,0.0458
2,0,0,"(Fp1, P8, Fz)",0.50,0,6,0.7930,0.1092
3,0,0,"(Fp1, P8, Fz)",0.50,0,12,0.7930,0.1092
4,0,0,"(Fp1, P8, Fz)",0.50,0,24,0.7930,0.1092
5,0,0,"(Fp2, O1, T7)",0.01,0,6,0.7863,0.1210
6,0,0,"(Fp2, O1, T7)",0.01,0,12,0.7863,0.1210
7,0,0,"(Fp2, O1, T7)",0.01,0,24,0.7863,0.1210
8,0,0,"(Fp2, O1, T7)",0.01,1,6,0.7863,0.1210
9,0,0,"(Fp2, O1, T7)",0.01,1,12,0.7863,0.1210


In [ ]:
# Código anterior

# list_results = []
# for n_prep, prep in enumerate(lista_new_preproceso):
#     for n_troz, troz in enumerate(prep):
#         for n_chan, mat in enumerate(troz):
#             for eta in [0.01, 0.1, 0.5, 1]:
#                 for gamma in [0, 1]:
#                     for max_depth in [2, 3, 6, 12, 24]:
#                         print('prep:',n_prep,'troz:', n_troz,'chan:', n_chan,'eta:', eta,'gamma:', gamma,'max_depth:', max_depth)
#                         # Cogemos los datos
#                         data_dm = mat.iloc[:,:-1].to_numpy()
#                         label_dm = mat['Label'].to_numpy()
#                         dtrain = xgb.DMatrix(data_dm, label=label_dm)
#                         param = {"max_depth": max_depth, "eta": eta , "gamma": gamma, "objective": "binary:logistic"}
#                         num_round = 30
#                         k_folds = StratifiedKFold(n_splits=5)
#                         res = xgb.cv(
#                                 param,
#                                 dtrain,
#                                 num_round,
#                                 stratified=True,
#                                 folds= k_folds,
#                                 metrics={"error"},
#                                 seed=0,
#                                 verbose_eval=0
#                             )
#                         min_row = res[res['test-error-mean'] == res['test-error-mean'].min()].iloc[0,:]
#                         min_error = round(min_row.iloc[2],4)
#                         std_error = round(min_row.iloc[3], 4)
#                         list_results.append((n_prep, n_troz, n_chan, eta, gamma, max_depth, 1-min_error, std_error))
#                         print('Accuracy máximo obtenido:', 1-min_error, '+-', std_error)
